In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
proc = pd.read_csv('data/procedures.csv')
phy = pd.read_csv('data/physicians.csv')

In [3]:
phy['Cardiologist'] = pd.get_dummies(phy.specialty)['Cardiology']

In [4]:
phy.columns = ['physician_id','specialty','Cardiologist']

In [5]:
merged = pd.merge(proc, phy, left_on='physician_id',right_on='physician_id',how='outer')
indices=[]
for i,proc in enumerate(merged.procedure):
    w=[]
    w.append(proc.replace('_', ' '))
    if 'typically' in w[0].split():
        indices.append(i)

In [6]:
merged_not_typically = merged.drop(merged.index[list(indices)])
merged_not_typically = merged_not_typically.reset_index()
merged_not_typically.drop('index',axis=1,inplace=True)

In [7]:
merged_not_typically_no_un = merged_not_typically[merged_not_typically.specialty!='Unknown']


In [8]:
d={}
for i,proc in enumerate(merged_not_typically_no_un.procedure_code):
    if proc not in d.keys():
        d[proc] = [merged_not_typically_no_un.specialty.iloc[i]]
    if proc in d.keys():
        d[proc].append(merged_not_typically_no_un.specialty.iloc[i])
        

In [9]:
merged_not_typically_only_un = merged_not_typically[merged_not_typically.specialty=='Unknown']

In [10]:
merged_not_typically_only_un.reset_index(drop=True, inplace=True)

In [11]:
merged_not_typically.head(10)

,physician_id,procedure_code,procedure,number_of_patients,specialty,Cardiologist
0,1,76942,ultrasonic_guidance_imaging_supervision_and_in...,14,Unknown,0
1,2,99238,"hospital_discharge_day_management,_30_minutes_...",16,Family Practice,0
2,3,99283,"emergency_department_visit,_moderately_severe_...",68,Emergency Medicine,0
3,3,99284,"emergency_department_visit,_problem_of_high_se...",87,Emergency Medicine,0
4,3,99285,"emergency_department_visit,_problem_with_signi...",267,Emergency Medicine,0
5,3,99291,critical_care_delivery_critically_ill_or_injur...,15,Emergency Medicine,0
6,5,36247,insertion_of_catheter_into_abdominal_pelvic_or...,15,Cardiology,1
7,5,75625,radiological_supervision_and_interpretation_x-...,20,Cardiology,1
8,5,75716,radiological_supervision_and_interpretation_of...,17,Cardiology,1
9,5,78492,nuclear_medicine_study_heart_muscle_at_rest_an...,45,Cardiology,1


In [12]:
len(merged_not_typically)

416588

In [15]:
merged_not_typically.physician_id.iloc[9]

5

In [13]:
g = nx.DiGraph()

In [16]:
for i in range(len(merged_not_typically)):
    g.add_edge(merged_not_typically.physician_id.iloc[i],merged_not_typically.procedure_code.iloc[i],weight=merged_not_typically.number_of_patients.iloc[i])

In [18]:
g.number_of_edges()

398020

In [20]:
g.number_of_nodes()

36175

In [23]:
g.size()

398020

In [25]:
g[1]['76942']

{'weight': 14}

In [35]:
nx.write_gexf(g,'weighted.gexf')